In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns

from copy import deepcopy


#For random forest for time series:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

#For metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("Single-House-Optimization.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")

#From Functions folder
from LoadSeries import load_series, moving_average
from Merge import merge
from Battery import Battery
from DPModel import DP, DP_carb
from Logic import logic_bat
from Funcs_Logic_DP import action_rollout, pred_logic_rollout, print_price_summary, logic_series_print
from MPC import MPC, MPC_carb, MPCModel

In [2]:
house = "h16"
prod = load_series("p",house,"prod_"+house,[24,48,168])
cons = load_series("c",house,"cons_"+house,[24,48,168])
cons

,cons_h16,cons_h16-24,cons_h16-48,cons_h16-168,Year,Month,Day,Weekday,Hour
2020-12-29 00:00:00,5.7,5.7,6.9,6.3,2020,12,29,1,0
2020-12-29 01:00:00,5.9,5.4,5.3,4.8,2020,12,29,1,1
2020-12-29 02:00:00,5.0,6.0,4.9,4.8,2020,12,29,1,2
2020-12-29 03:00:00,3.9,6.8,4.4,4.3,2020,12,29,1,3
2020-12-29 04:00:00,3.5,4.0,3.6,4.0,2020,12,29,1,4
...,...,...,...,...,...,...,...,...,...
2023-01-10 16:00:00,6.1,7.6,6.9,5.4,2023,1,10,1,16
2023-01-10 17:00:00,8.3,10.0,9.8,6.2,2023,1,10,1,17
2023-01-10 18:00:00,10.4,9.1,14.5,13.0,2023,1,10,1,18
2023-01-10 19:00:00,9.9,11.1,10.8,10.4,2023,1,10,1,19


In [3]:
merged = merge(house)
merged

,prod_h16,cons_h16,yield,SpotPriceDKK,CO2Emission
2020-12-22 00:00:00,0.0,6.3,-6.3,11.090000,60.0
2020-12-22 01:00:00,0.0,4.8,-4.8,-0.150000,58.0
2020-12-22 02:00:00,0.0,4.8,-4.8,-55.880001,65.0
2020-12-22 03:00:00,0.0,4.3,-4.3,-72.919998,67.0
2020-12-22 04:00:00,0.0,4.0,-4.0,-53.720001,76.0
...,...,...,...,...,...
2022-12-31 19:00:00,0.0,8.7,-8.7,498.329987,209.0
2022-12-31 20:00:00,0.0,6.9,-6.9,301.190002,224.0
2022-12-31 21:00:00,0.0,5.3,-5.3,110.730003,253.0
2022-12-31 22:00:00,0.0,5.6,-5.6,73.919998,272.0


In [22]:
bat_capacity=13
stepsize=2
Start="2022-06-19 00:00:00"


train_prod = 4000
train_cons = 500

BeforeProd = pd.date_range(end=Start, periods=train_prod+1, freq="h")[0]
BeforeCons = pd.date_range(end=Start, periods=train_cons+1, freq="h")[0]

using_N=False

if using_N:
    N = 24*7
    End = pd.date_range(start=Start, periods=N, freq="h")[-1]

else:
    End = "2022-06-25 23:00:00"
    N=len(pd.date_range(start=Start,end=End,freq="h"))
    
remainder = N%24

if remainder==0:
    print(f"Period is of size {N} hours = {N//24} days\nPeroid from {Start} to {End}")
elif remainder==1:
    print(f"Period of size {N} hours = {N//24} days and {remainder} hour\nPeroid from {Start} to {End}")
else:
    print(f"Period of size {N} hours = {N//24} days and {remainder} hours\nPeroid from {Start} to {End}")

Period of size 7 hours = 0 days and 7 hours
Peroid from 2022-06-19 00:00:00 to 2022-06-19 06:00:00


## Running all models

In [23]:
forecasterProd = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123,max_depth=12,max_features='log2',n_estimators=412,n_jobs=4),
                 lags      = list(range(1,24+1,1))
                )
forecasterCons = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123,max_depth=9,max_features='log2',n_estimators=633,n_jobs=4),
                 lags      = list(range(1,12+1,1))
                )

battery_no   = Battery(max_capacity=bat_capacity,max_charge=0)
battery      = Battery(max_capacity=bat_capacity,max_charge=7)
DPbat_price  = Battery(max_capacity=bat_capacity,max_charge=7)
DPbat_carb   = Battery(max_capacity=bat_capacity,max_charge=7)
MPCbat_price = Battery(max_capacity=bat_capacity,max_charge=7)
MPCbat_carb  = Battery(max_capacity=bat_capacity,max_charge=7)

Mpc = MPCModel(house, 0.1, 0.0, 1, MPCbat_price.max_charge, MPCbat_price.max_capacity)


series_no_battery  = pd.DataFrame(columns=merged.columns)
series_battery     = pd.DataFrame(columns=merged.columns)
series_battery_DP_price  = pd.DataFrame(columns=merged.columns)
series_battery_DP_carb   = pd.DataFrame(columns=merged.columns)
series_battery_MPC_price  = pd.DataFrame(columns=merged.columns)
series_battery_MPC_carb   = pd.DataFrame(columns=merged.columns)

preds = pd.DataFrame() 

Start_i = Start
BeforeProd_i = BeforeProd
BeforeCons_i = BeforeCons

num_loops = int(np.ceil(N/stepsize))
remainder = N%stepsize
length = stepsize
for i in range(num_loops):
    if i == num_loops-1:
        length = length if remainder == 0 else remainder
            
    End_i = pd.date_range(start=Start_i,periods=24,freq="h")[-1]
    
    print(f"Loop number {i+1} of {num_loops}")
    print(f"Predicting period from {Start_i} to {End_i}")
    
    _, predProd = backtesting_forecaster(forecaster = forecasterProd,
                                         y          = prod.loc[BeforeProd_i:End_i]['prod_'+house],
                                         exog       = prod.loc[BeforeProd_i:End_i].drop(['prod_'+house], axis=1),
                                         initial_train_size = train_prod,
                                         steps      = 24,
                                         refit      = True,
                                         metric     = 'mean_squared_error',
                                         fixed_train_size   = True,
                                         verbose    = False
                                        )

    _, predCons = backtesting_forecaster(forecaster = forecasterCons,
                                         y          = cons.loc[BeforeCons_i:End_i]['cons_'+house],
                                         exog       = cons.loc[BeforeCons_i:End_i].drop(['cons_'+house], axis=1),
                                         initial_train_size = train_cons,
                                         steps      = 24,
                                         refit      = True,
                                         metric     = 'mean_squared_error',
                                         fixed_train_size   = True,
                                         verbose    = False
                                        )
    pred_i = pd.merge(predProd, predCons, how="outer", left_index=True,right_index=True)
    pred_i = pred_i.rename(columns={"pred_x":"prod_"+house,"pred_y":"cons_"+house})

    pred_i["prod_"+house]=np.array((pred_i["prod_"+house].to_numpy())*10,dtype=int)/10
    pred_i["cons_"+house]=np.array((pred_i["cons_"+house].to_numpy())*10,dtype=int)/10

    pred_i["yield"] = pred_i["prod_"+house] - pred_i["cons_"+house]
    
    values = pred_i["yield"].to_numpy()
    
    for j in range(len(values)):
        values[j] = f"{values[j]:.1f}"
    
    pred_i["yield"] = values
    
    
    pred_i["SpotPriceDKK"] = merged.loc[Start_i:End_i]["SpotPriceDKK"]
    pred_i["CO2Emission"] = merged.loc[Start_i:End_i]["CO2Emission"]
    
    merged_i = merged.loc[Start_i:End_i]
    
    print(f"Taking actions the first {length} actions from {Start_i} to {pred_i.index[length-1]}")
    
    temp_no_bat = pred_logic_rollout(merged_i, pred_i, deepcopy(battery_no), logic_bat)
    temp_bat = pred_logic_rollout(merged_i, pred_i, deepcopy(battery), logic_bat)
    
    print("\rRunnig DP price optimization",end="")
    actions_DP_price = DP(Start_i,End_i,house,pred_i,deepcopy(DPbat_price),byday=True,ints=True,degrade=False,verbose=False)
    print("\r                                        ",end="")
    print("\rRunnig DP carbon emissions optimization",end="")
    actions_DP_carb  = DP_carb(Start_i,End_i,house,pred_i,deepcopy(DPbat_carb),byday=True,ints=True,degrade=False,verbose=False)
    print("\r                                        ",end="")
    print("\rRunnig MPC price optimization",end="")
    actions_MPC_price = MPC(Start_i,End_i,pred_i,deepcopy(MPCbat_price),Mpc,byday=True,verbose=False)
    print("\r                                        ",end="")
    print("\rRunnig MPC carbon emissions optimization",end="")
    actions_MPC_carb  = MPC_carb(Start_i,End_i,pred_i,deepcopy(MPCbat_carb),Mpc,byday=True,verbose=False)
    print("\r                                        ",end="")
    print("\rDone!",end="")
    
    series_no_battery_i         = action_rollout(merged_i.iloc[:length], battery_no, temp_no_bat[:length])
    series_battery_i            = action_rollout(merged_i.iloc[:length], battery, temp_bat[:length])
    series_battery_DP_price_i   = action_rollout(merged_i.iloc[:length], DPbat_price, actions_DP_price)
    series_battery_DP_carb_i    = action_rollout(merged_i.iloc[:length], DPbat_carb, actions_DP_carb)
    series_battery_MPC_price_i  = action_rollout(merged_i.iloc[:length], MPCbat_price, actions_MPC_price)
    series_battery_MPC_carb_i   = action_rollout(merged_i.iloc[:length], MPCbat_carb, actions_MPC_carb)
    
    
    Start_i = pd.date_range(start=Start_i,periods=length+1,freq="h")[-1]
    BeforeProd_i = pd.date_range(start=BeforeProd_i,periods=length+1,freq="h")[-1]
    BeforeCons_i = pd.date_range(start=BeforeCons_i,periods=length+1,freq="h")[-1]
    
    preds=pd.concat([preds,pred_i[:length]])
    series_no_battery  = pd.concat([series_no_battery,series_no_battery_i])
    series_battery     = pd.concat([series_battery,series_battery_i])
    series_battery_DP_price  = pd.concat([series_battery_DP_price,series_battery_DP_price_i])
    series_battery_DP_carb   = pd.concat([series_battery_DP_carb,series_battery_DP_carb_i])
    series_battery_MPC_price  = pd.concat([series_battery_MPC_price,series_battery_MPC_price_i])
    series_battery_MPC_carb   = pd.concat([series_battery_MPC_carb,series_battery_MPC_carb_i])
    
    print()
    print()

    
series_no_battery["cost_cummulative"] = series_no_battery["cost"].cumsum(axis=0)
series_battery["cost_cummulative"] = series_battery["cost"].cumsum(axis=0) 
series_battery_DP_price["cost_cummulative"] = series_battery_DP_price["cost"].cumsum(axis=0)
series_battery_DP_carb["cost_cummulative"] = series_battery_DP_carb["cost"].cumsum(axis=0)
series_battery_MPC_price["cost_cummulative"] = series_battery_MPC_price["cost"].cumsum(axis=0)
series_battery_MPC_carb["cost_cummulative"] = series_battery_MPC_carb["cost"].cumsum(axis=0)


series_no_battery["emission_cummulative"] = series_no_battery["emission"].cumsum(axis=0)
series_battery["emission_cummulative"] = series_battery["emission"].cumsum(axis=0) 
series_battery_DP_price["emission_cummulative"] = series_battery_DP_price["emission"].cumsum(axis=0)
series_battery_DP_carb["emission_cummulative"] = series_battery_DP_carb["emission"].cumsum(axis=0)
series_battery_MPC_price["emission_cummulative"] = series_battery_MPC_price["emission"].cumsum(axis=0)
series_battery_MPC_carb["emission_cummulative"] = series_battery_MPC_carb["emission"].cumsum(axis=0)
preds

Loop number 1 of 3
Predicting period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Taking actions the first 3 actions from 2022-06-19 00:00:00 to 2022-06-19 02:00:00
Done!                                   

Loop number 2 of 3
Predicting period from 2022-06-19 03:00:00 to 2022-06-20 02:00:00
Taking actions the first 3 actions from 2022-06-19 03:00:00 to 2022-06-19 05:00:00
Done!                                   

Loop number 3 of 3
Predicting period from 2022-06-19 06:00:00 to 2022-06-20 05:00:00
Taking actions the first 1 actions from 2022-06-19 06:00:00 to 2022-06-19 06:00:00
Done!                                   



,prod_h16,cons_h16,yield,SpotPriceDKK,CO2Emission
2022-06-19 00:00:00,0.0,6.9,-6.9,1540.390015,137.0
2022-06-19 01:00:00,0.0,6.5,-6.5,1413.410034,102.0
2022-06-19 02:00:00,0.0,5.3,-5.3,1231.300049,146.0
2022-06-19 03:00:00,0.0,3.9,-3.9,1039.969971,155.0
2022-06-19 04:00:00,0.0,3.7,-3.7,897.140015,155.0
2022-06-19 05:00:00,0.0,3.5,-3.5,825.429993,185.0
2022-06-19 06:00:00,0.5,3.4,-2.9,837.260010,241.0


## No battery

In [24]:
print_price_summary(series_no_battery)
logic_series_print(series_no_battery)

The period is from 2022-06-19 00:00:00 to 2022-06-19 06:00:00
Cost for period:  34.0  DKK
Total emissions for period:  5.0  kg
Average cost per year is: 50014.0 DKK
Average emissions per year is: 6659.0 kg
Number of kwh purchased in the period: 29.5
Average number of kwh purchased per year: 43099.5
Average number of kwh sold per year: 0.0
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:   0.8971,  0.1550, -3.8000, -3

## Simple logic

In [25]:
print_price_summary(series_battery)
logic_series_print(series_battery)

The period is from 2022-06-19 00:00:00 to 2022-06-19 06:00:00
Cost for period:  34.0  DKK
Total emissions for period:  5.0  kg
Average cost per year is: 50014.0 DKK
Average emissions per year is: 6659.0 kg
Number of kwh purchased in the period: 29.5
Average number of kwh purchased per year: 43099.5
Average number of kwh sold per year: 0.0
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:   0.8971,  0.1550, -3.8000, -3

## DP

In [26]:
print_price_summary(series_battery_DP_price)
logic_series_print(series_battery_DP_price)

The period is from 2022-06-19 00:00:00 to 2022-06-19 06:00:00
Cost for period:  34.0  DKK
Total emissions for period:  4.0  kg
Average cost per year is: 49964.0 DKK
Average emissions per year is: 6422.0 kg
Number of kwh purchased in the period: 29.500000000000004
Average number of kwh purchased per year: 43099.50000000001
Average number of kwh sold per year: 0.0
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:   0.89

In [27]:
print_price_summary(series_battery_DP_carb)
logic_series_print(series_battery_DP_carb)

The period is from 2022-06-19 00:00:00 to 2022-06-19 06:00:00
Cost for period:  55.0  DKK
Total emissions for period:  6.0  kg
Average cost per year is: 80655.0 DKK
Average emissions per year is: 8903.0 kg
Number of kwh purchased in the period: 45.5
Average number of kwh purchased per year: 66475.5
Average number of kwh sold per year: 4383.0
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000,-11.5000,  6.0000,  6.0000,  0.0000,  0.0000,  6.0000, 17.7145, 17.7145,  1.5755,  1.5755
    1:   1.4134,  0.1020, -5.3000,-12.3000,  7.0000,  7.0000,  6.0000,  6.0000, 13.0000, 17.3849, 35.0994,  1.2546,  2.8301
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000, -0.0000, 13.0000, 13.0000, 13.0000,  5.1715, 40.2709,  0.6132,  3.4433
    3:   1.0400,  0.1550, -3.8000,  3.0000,  0.0000, -6.8000, 13.0000, 13.0000,  6.2000, -0.3120, 39.9589, -0.4650,  2.9783
    4:   0.8971,  0.1550, -3.8000,

## MPC

In [28]:
print_price_summary(series_battery_MPC_price)
logic_series_print(series_battery_MPC_price)

The period is from 2022-06-19 00:00:00 to 2022-06-19 06:00:00
Cost for period:  34.0  DKK
Total emissions for period:  4.0  kg
Average cost per year is: 49964.0 DKK
Average emissions per year is: 6422.0 kg
Number of kwh purchased in the period: 29.500000000000004
Average number of kwh purchased per year: 43099.50000000001
Average number of kwh sold per year: 0.0
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:   0.89

In [29]:
print_price_summary(series_battery_MPC_carb)
logic_series_print(series_battery_MPC_carb)

The period is from 2022-06-19 00:00:00 to 2022-06-19 06:00:00
Cost for period:  53.0  DKK
Total emissions for period:  6.0  kg
Average cost per year is: 77346.0 DKK
Average emissions per year is: 8903.0 kg
Number of kwh purchased in the period: 42.5
Average number of kwh purchased per year: 62092.5
Average number of kwh sold per year: 0.0
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000,-11.5000,  6.0000,  6.0000,  0.0000,  0.0000,  6.0000, 17.7145, 17.7145,  1.5755,  1.5755
    1:   1.4134,  0.1020, -5.3000,-12.3000,  7.0000,  7.0000,  6.0000,  6.0000, 13.0000, 17.3849, 35.0994,  1.2546,  2.8301
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000, -0.0000, 13.0000, 13.0000, 13.0000,  5.1715, 40.2709,  0.6132,  3.4433
    3:   1.0400,  0.1550, -3.8000, -0.8000,  0.0000, -3.0000, 13.0000, 13.0000, 10.0000,  0.8320, 41.1029,  0.1240,  3.5673
    4:   0.8971,  0.1550, -3.8000, -6